In [3]:
import pandas as pd
from catboost import CatBoostClassifier
from datetime import date, timedelta
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm

In [4]:
df = pd.read_csv("/dataset.csv")

df["partnerrolestart_date"] = df["partnerrolestart_date"].astype("datetime64[ns]")
df["client_start_date"] = df["client_start_date"].astype("datetime64[ns]")

cutoff_train_date = pd.to_datetime(date(2020, 9, 1))
cutoff_test_date = pd.to_datetime(date(2020, 12, 1))
min_date = df["client_start_date"].min()
num_weeks = (cutoff_train_date - min_date).days // 7 + 1

Дальше идёт создание как раз-таки датасета и генерация признаков

In [5]:
def weeks_count(lst_dates: list) -> pd.Series:
    lst_weeks = [0 for _ in range(num_weeks)]
    for item in lst_dates:
        lst_weeks[(item - min_date).days // 7] += 1
    return pd.Series(lst_weeks)


def max_length(row: pd.Series) -> int:
    row = row.loc['week_1':'week_79']
    counter_max = 0
    counter = 0
    for element in row:
        if element > 0:
            counter += 1
        counter_max = max(counter, counter_max)
        if element == 0:
            counter = 0
    return counter_max


def max_length_increasing(row: pd.Series) -> int:
    row = row.loc['week_1':'week_79']
    counter_max = 0
    counter = 0
    prev_sequence = [1]
    for element in row:
        if element >= max(prev_sequence):
            counter += 1
        counter_max = max(counter, counter_max)
        if element < max(prev_sequence):
            counter = 0
            prev_sequence = [1]
        prev_sequence.append(element)
    return counter_max


def max_length_zeros(row: pd.Series) -> int:
    row = row.loc['week_1':'week_79']
    counter_max = 0
    counter = 0
    for element in row:
        if element == 0:
            counter += 1
        counter_max = max(counter, counter_max)
        if element != 0:
            counter = 0
    return counter_max

def num_nunzero_streaks(row: pd.Series) -> int:
    row = row.loc['week_1':'week_79']
    counter = 0
    lst_nnz = []
    for element in row:
        if element != 0:
            lst_nnz.append(element)
        else:
            counter += (len(lst_nnz) >= 5)
            lst_nnz = []
    return counter


def recent_clients(row: pd.Series) -> int:
    row = row.loc['week_70':'week_79']
    return sum(row)

In [6]:
df_train_cutoff = df[df["client_start_date"] < cutoff_train_date]
data_train = (
    df_train_cutoff.groupby("clientbankpartner_pin")
    .agg(
        {
            "client_pin": "count",
            "partner_src_type_ccode": "first",
            "partnerrolestart_date": "first",
            "client_start_date": lambda x: list(x),
        }
    )
    .reset_index()
)

data_train.rename(
    {
        "clientbankpartner_pin": "partner_pin",
        "client_pin": "num_clients",
        "partner_src_type_ccode": "partner_type",
        "partnerrolestart_date": "partner_start_date",
        "client_start_date": "lst_dates_involving",
    },
    axis=1,
    inplace=True,
)

time_series_train = data_train["lst_dates_involving"].apply(weeks_count)
time_series_train.columns = [f"week_{i+1}" for i in range(79)]
data_train = pd.concat([data_train, time_series_train], axis=1)

data_train = data_train.assign(
    time_as_partner=(cutoff_train_date - data_train["partner_start_date"]).dt.days,
    since_last_client=data_train["lst_dates_involving"].apply(lambda x: (cutoff_train_date - max(x)).days),
    max_length=data_train.apply(max_length, axis=1),
    max_length_increasing=data_train.apply(max_length_increasing, axis=1),
    max_length_zeros=data_train.apply(max_length_zeros, axis=1),
    recent_clients=data_train.apply(recent_clients, axis=1)
)

Тут идёт трейн)

In [7]:
positive_scores = df[df['client_start_date'] >= cutoff_train_date]['clientbankpartner_pin'].unique()
train_score = data_train['partner_pin'].apply(lambda x: 0 if x in positive_scores else 1)
drop_on_fitting = {'columns': ["partner_pin", "lst_dates_involving", "partner_start_date"]}

X_train, X_val, y_train, y_val = train_test_split(data_train.drop(**drop_on_fitting), train_score, train_size=0.8, random_state=42)

In [8]:
model_params = {
    "iterations": 1000,
    "learning_rate": 0.03,
    "depth": 3,
    "l2_leaf_reg": 5,
    "bootstrap_type": "Bernoulli",
    "subsample": 0.8,
    "random_seed": 42,
    "eval_metric": "AUC",
    "early_stopping_rounds": 100,
    "verbose": True,
}

model_train = CatBoostClassifier(**model_params).fit(X_train, y_train)
roc_auc_score(y_val, model_train.predict(X_val))

0:	total: 62.9ms	remaining: 1m 2s
1:	total: 67.4ms	remaining: 33.6s
2:	total: 71.6ms	remaining: 23.8s
3:	total: 75.7ms	remaining: 18.9s
4:	total: 79.5ms	remaining: 15.8s
5:	total: 83.5ms	remaining: 13.8s
6:	total: 87.7ms	remaining: 12.4s
7:	total: 91.4ms	remaining: 11.3s
8:	total: 95.5ms	remaining: 10.5s
9:	total: 99ms	remaining: 9.8s
10:	total: 102ms	remaining: 9.16s
11:	total: 105ms	remaining: 8.63s
12:	total: 108ms	remaining: 8.2s
13:	total: 111ms	remaining: 7.83s
14:	total: 115ms	remaining: 7.52s
15:	total: 118ms	remaining: 7.24s
16:	total: 121ms	remaining: 6.99s
17:	total: 124ms	remaining: 6.79s
18:	total: 128ms	remaining: 6.6s
19:	total: 131ms	remaining: 6.43s
20:	total: 135ms	remaining: 6.28s
21:	total: 138ms	remaining: 6.12s
22:	total: 141ms	remaining: 5.98s
23:	total: 144ms	remaining: 5.85s
24:	total: 147ms	remaining: 5.74s
25:	total: 150ms	remaining: 5.62s
26:	total: 154ms	remaining: 5.53s
27:	total: 157ms	remaining: 5.45s
28:	total: 160ms	remaining: 5.37s
29:	total: 164ms	re

0.764241121860481

Теперь идём тестировать!!! (и послыать)

In [12]:
def weeks_count_full(lst_dates: list) -> pd.Series:
    lst_weeks = [0 for _ in range(num_weeks + (cutoff_test_date - cutoff_train_date).days // 7)]
    for item in lst_dates:
        lst_weeks[(item - min_date).days // 7] += 1
    return pd.Series(lst_weeks[13:])

In [13]:
data_test = (
    df.groupby("clientbankpartner_pin")
    .agg(
        {
            "client_pin": "count",
            "partner_src_type_ccode": "first",
            "partnerrolestart_date": "first",
            "client_start_date": lambda x: list(x),
        }
    )
    .reset_index()
)

data_test.rename(
    {
        "clientbankpartner_pin": "partner_pin",
        "client_pin": "num_clients",
        "partner_src_type_ccode": "partner_type",
        "partnerrolestart_date": "partner_start_date",
        "client_start_date": "lst_dates_involving",
    },
    axis=1,
    inplace=True,
)

time_series_test = data_test["lst_dates_involving"].apply(weeks_count_full)
time_series_test.columns = [f"week_{i+1}" for i in range(79)]
data_test = pd.concat([data_test, time_series_test], axis=1)

data_test = data_test.assign(
    time_as_partner=(cutoff_test_date - data_test["partner_start_date"]).dt.days,
    since_last_client=data_test["lst_dates_involving"].apply(
        lambda x: (cutoff_test_date - max(x)).days
    ),
    max_length=data_test.apply(max_length, axis=1),
    max_length_increasing=data_test.apply(max_length_increasing, axis=1),
    max_length_zeros=data_test.apply(max_length_zeros, axis=1),
    recent_clients=data_test.apply(recent_clients, axis=1),
)

model = CatBoostClassifier(**model_params).fit(data_train.drop(**drop_on_fitting), train_score)
data_test["score"] = model.predict_proba(data_test.drop(**drop_on_fitting).astype(float))[:, 1]

0:	total: 4.71ms	remaining: 4.71s
1:	total: 8.79ms	remaining: 4.39s
2:	total: 12.2ms	remaining: 4.05s
3:	total: 16.1ms	remaining: 4.01s
4:	total: 19.6ms	remaining: 3.91s
5:	total: 23.4ms	remaining: 3.88s
6:	total: 26.9ms	remaining: 3.82s
7:	total: 30.7ms	remaining: 3.81s
8:	total: 35ms	remaining: 3.86s
9:	total: 38.9ms	remaining: 3.85s
10:	total: 43.4ms	remaining: 3.9s
11:	total: 47.3ms	remaining: 3.9s
12:	total: 51.6ms	remaining: 3.92s
13:	total: 56.2ms	remaining: 3.96s
14:	total: 61ms	remaining: 4s
15:	total: 66.7ms	remaining: 4.1s
16:	total: 71.7ms	remaining: 4.14s
17:	total: 75.8ms	remaining: 4.13s
18:	total: 80.6ms	remaining: 4.16s
19:	total: 84.7ms	remaining: 4.15s
20:	total: 89.1ms	remaining: 4.16s
21:	total: 93.1ms	remaining: 4.14s
22:	total: 97.6ms	remaining: 4.14s
23:	total: 101ms	remaining: 4.12s
24:	total: 105ms	remaining: 4.11s
25:	total: 109ms	remaining: 4.09s
26:	total: 114ms	remaining: 4.09s
27:	total: 117ms	remaining: 4.08s
28:	total: 122ms	remaining: 4.08s
29:	total: 

Сабмитный файл:

In [14]:
sub = pd.read_csv('/submission_example.csv')
sub['score'] = sub['clientbankpartner_pin'].apply(lambda x: data_test[data_test['partner_pin'] == x]['score'].iloc[0])
sub.to_csv('BaseSub_Infer.csv', index=False)
sub

clientbankpartner_pin     score
0                      6781  0.023821
1                    236905  0.027033
2                    125779  0.194247
3                      1952  0.261018
4                      4872  0.412947
...                     ...       ...
4503                 121120  0.264987
4504                  60667  0.680655
4505                   5065  0.667487
4506                 133125  0.363005
4507                 121963  0.536567

[4508 rows x 2 columns]